In [1]:
import numpy as np 
import pandas as pd 
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/emnlp-dataset-2025/English-Mizo Traning Data 2025.xlsx
/kaggle/input/emnlp-dataset-2025/English-Nyishi Training Data 2025.xlsx
/kaggle/input/emnlp-dataset-2025/English-Assamese Training Data 2025.csv
/kaggle/input/emnlp-dataset-2025/English-Manipuri Training Data 2025.xlsx
/kaggle/input/emnlp-dataset-2025/English-Khasi Training Data 2025.xlsx
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Mizo/en-lus.txt
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Mizo/lus-en.txt
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Assamese/as-en.txt
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Assamese/en-as.txt
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Manipuri/en-mni.txt
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Manipuri/mni-en.txt
/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Cat

In [2]:
!pip install -q transformers datasets accelerate bitsandbytes peft evaluate sacrebleu
!pip install -U bitsandbytes
!pip install datasets pandas sacremoses fasttext sentence-transformers sentencepiece scikit-learn
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
!pip install spacy indic-transliteration
!python -m spacy download en_core_web_sm 
!pip install indic-nlp-library 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, TaskType, get_peft_model
import evaluate 

2025-06-09 19:18:11.102796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749496691.286773      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749496691.341712      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from datasets import load_dataset, Dataset

# Load original CSV
dataset = load_dataset(
    'csv',
    data_files="/kaggle/input/emnlp-dataset-2025/English-Assamese Training Data 2025.csv",
    split="train"
)

# Reverse columns: Assamese ← English, English ← Assamese
reversed_examples = [{"as": example["as"], "en": example["en"]} for example in dataset]

# Rebuild as a new Dataset with exactly ordered columns
reversed_dataset = Dataset.from_list(reversed_examples)

# Shuffle and take 100 rows
subset = reversed_dataset.shuffle(seed=42).select(range(20000))

# Train-test split
dataset_split = subset.train_test_split(test_size=0.05)
train_dataset = dataset_split["train"]
test_dataset = dataset_split["test"]

# Confirm column order
print(train_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['as', 'en'],
    num_rows: 900
})


In [5]:
test_dataset

Dataset({
    features: ['as', 'en'],
    num_rows: 100
})

In [6]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("as")

def normalize_assamese(batch):
    return {'as': [normalizer.normalize(x) for x in batch['as']], 'en': batch['en']}

train_dataset = train_dataset.map(normalize_assamese, batched=True)
test_dataset=test_dataset.map(normalize_assamese, batched=True)

def remove_duplicates(batch):
    seen = set()
    filt = []
    for en, as_ in zip(batch['en'], batch['as']):
        key = (en, as_)
        if key not in seen:
            seen.add(key)
            filt.append(True)
        else:
            filt.append(False)
    return {"keep": filt}

train_dataset = train_dataset.map(remove_duplicates, batched=True).filter(lambda x: x['keep'])
test_dataset = test_dataset.map(remove_duplicates, batched=True).filter(lambda x: x['keep'])

print(f"After removing duplicates: {len(train_dataset)}")
train_dataset[:5]

def ratio_filter(en, as_):
    ratio = len(en.split()) / max(1, len(as_.split()))
    return 0.5 <= ratio <= 2.0

train_dataset = train_dataset.filter(lambda x: ratio_filter(x['en'], x['as']))
test_dataset = test_dataset.filter(lambda x: ratio_filter(x['en'], x['as']))

import string

def preprocess_text(batch):
    # Create a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    
    # Process English text
    processed_en = [text.translate(translator) for text in batch['en']]
    
    # Process Assamese text (assuming 'as' is the key for Assamese text)
    processed_as = [text.lower().translate(translator) for text in batch['as']]
    
    return {'en': processed_en, 'as': processed_as}

# Then apply text preprocessing
train_dataset = train_dataset.map(preprocess_text, batched=True)
test_dataset = test_dataset.map(preprocess_text, batched=True)

print(f"After preprocessing: {len(train_dataset)}")
train_dataset[:5]

import unicodedata

def to_halfwidth(text):
    return unicodedata.normalize('NFKC', text)

train_dataset = train_dataset.map(lambda x: {
    'en': to_halfwidth(x['en']),
    'as': to_halfwidth(x['as'])
})
test_dataset = test_dataset.map(lambda x: {
    'en': to_halfwidth(x['en']),
    'as': to_halfwidth(x['as'])
})
print(test_dataset[:3])
print(train_dataset[:3])

import fasttext

# Make sure lid.176.ftz is in /kaggle/working or input path
ft_model = fasttext.load_model("lid.176.ftz")

def lang_filter(text, lang):
    pred = ft_model.predict(text.replace(' ', '_'), k=1)
    code = pred[0][0].replace('__label__', '')
    return code == lang

train_dataset = train_dataset.filter(lambda x: lang_filter(x['en'], 'en') and lang_filter(x['as'], 'as'))
test_dataset = test_dataset.filter(lambda x: lang_filter(x['en'], 'en') and lang_filter(x['as'], 'as'))

print(f"After language filtering: {len(train_dataset)}")
train_dataset[:3]

train_dataset = train_dataset.remove_columns('keep')
test_dataset = test_dataset.remove_columns('keep')

def length_filter(en, as_):
    return len(en.split()) <= 150 and len(as_.split()) <= 150

train_dataset = train_dataset.filter(lambda x: length_filter(x['en'], x['as']))
test_dataset = test_dataset.filter(lambda x: length_filter(x['en'], x['as']))

print(f"After length filtering: {len(train_dataset)}")
train_dataset[:3] 

import string

translator = str.maketrans('', '', string.punctuation)

def remove_punctuation(batch):
    return {
        'en': [text.translate(translator) for text in batch['en']],
        'as': [text.translate(translator) for text in batch['as']],
    }

# Apply to both train and test datasets
train_dataset = train_dataset.map(remove_punctuation, batched=True)
test_dataset = test_dataset.map(remove_punctuation, batched=True)

def clean_whitespace(batch):
    return {
        'en': [x.strip().replace("  ", " ") for x in batch['en']],
        'as': [x.strip().replace("  ", " ") for x in batch['as']]
    }

train_dataset = train_dataset.map(clean_whitespace, batched=True)
test_dataset=test_dataset.map(clean_whitespace,batched=True)

import pandas as pd

data = pd.DataFrame(train_dataset[:10])  # Load first 10 rows
print(data['en'][3])
print(data['as'][3])

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Filter:   0%|          | 0/900 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/689 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/689 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Does a grasshopper have cause to vaunt its prowess just because it can hop a little farther than other grasshoppers
এটা ফৰিঙে আনটো ফৰিঙৰ তুলনাত নিজকে মহান বুলি ভাবিব পাৰেনে


In [7]:
print("Train samples:")
for i in range(3):
    print(f"EN: {train_dataset[i]['en']}")
    print(f"AS: {train_dataset[i]['as']}")
    print()

Train samples:
EN: And when thy herds and thy flocks multiply and thy silver and thy gold is multiplied and all that thou hast is multiplied
AS: যেতিয়া আপোনালোকৰ গৰু ছাগলী ভেড়া জাকৰ বৃদ্ধি পাব আপোনালোকৰ অধিক সোণ ৰূপ হব আৰু আপোনালোকৰ সকলো সম্পত্তি বৃদ্ধি পাব

EN: That I may make it manifest as I ought to speak
AS: আপোনালোকৰ বাৰ্তালাপ যেন সদায় অনুগ্ৰহযুক্ত হৈ থাকক আৰু আস্বাদযুক্ত লোণৰদৰে কোন জনক কেনেকৈ উত্তৰ দিব লাগে সেই বিষয়ে যেন আপোনালোকে জানি লওঁক ।

EN: In the midst of material abundance many barely eke out a living
AS: বহুতে ভৌতিকভাৱে সমৃদ্ধিশীল হোৱা পৰিস্থিতিৰ মাজত থকাৰ স্বত্বেও জীয়াই থাকিবলৈ সাধাৰণভাৱে সংগ্ৰাম কৰিবলগীয়া হৈছে ।



In [8]:
train_dataset[0]

{'as': 'যেতিয়া আপোনালোকৰ গৰু ছাগলী ভেড়া জাকৰ বৃদ্ধি পাব আপোনালোকৰ অধিক সোণ ৰূপ হব আৰু আপোনালোকৰ সকলো সম্পত্তি বৃদ্ধি পাব',
 'en': 'And when thy herds and thy flocks multiply and thy silver and thy gold is multiplied and all that thou hast is multiplied'}

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

model_name = "facebook/nllb-200-3.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Set target language code (must match NLLB language codes)
tgt_lang = "eng_Latn"
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)

# Configure 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    forced_bos_token_id=forced_bos_token_id
)


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/94.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
def preprocess_examples(examples):
    # Set source and target languages
    tokenizer.src_lang = "asm_Beng"
    tokenizer.tgt_lang = "eng_Latn"

    # Tokenize inputs and targets
    model_inputs = tokenizer(
        examples["as"],
        max_length=256,
        padding="longest",
        truncation=True
    )
    
    labels = tokenizer(
        examples["en"],
        max_length=256,
        padding="longest",
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply to datasets
train_dataset = train_dataset.map(preprocess_examples, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess_examples, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/689 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [11]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  
    #use_dora=True
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
device = torch.device("cuda")

model=model.to(device)


trainable params: 56,623,104 || all params: 3,401,486,336 || trainable%: 1.6647


In [12]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    pad_to_multiple_of=8,
    label_pad_token_id=-100
)

In [13]:
bleu_metric = evaluate.load("sacrebleu")


In [14]:
import torch

# Sample Assamese input
assamese_text = "চি কৰ্পৰেছন আমেৰিকা যুক্তৰাষ্ট্ৰত প্ৰকল্প ৰূপায়নৰ হেতু আমেৰিকা যুক্তৰাষ্ট্ৰৰ টেক্সাছত গঠন কৰা হৈছে ।"

# Set tokenizer source and target language
tokenizer.src_lang = "asm_Beng"
tokenizer.tgt_lang = "eng_Latn"

# Tokenize the input
inputs = tokenizer(
    assamese_text,
    return_tensors="pt",
    padding=True,
    truncation=True
).to(model.device)

# Generate output
translated_tokens = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    max_length=256
)

# Decode the output
translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print("Translated Text:", translation)


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Translated Text: C Corporation was formed in Texas, United States, to implement projects in the United States.


In [15]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds.predictions, eval_preds.label_ids
    if isinstance(preds, tuple):
        preds = preds[0]
    if preds.ndim == 3:
        preds = np.argmax(preds, axis=-1)

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
   # decoded_preds = [transliterate(p, sanscript.DEVANAGARI, sanscript.BENGALI) for p in decoded_preds]
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}


In [16]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=3e-5, 
    warmup_steps=1000,
    lr_scheduler_type="cosine", 
    logging_steps=500,
    eval_strategy="epoch",
    save_strategy="epoch",
   # save_steps=1000,
    save_total_limit=2,
    fp16=True,
    predict_with_generate=True,
    report_to="none",
    optim="adamw_torch_fused",
    adam_beta1=0.9,
    adam_beta2=0.98,
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

model.config.use_cache = False

trainer.train()

/tmp/ipykernel_35/4153509896.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Bleu
0,No log,7.740397,22.442386


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


TrainOutput(global_step=43, training_loss=9.732076955396076, metrics={'train_runtime': 202.1526, 'train_samples_per_second': 3.408, 'train_steps_per_second': 0.213, 'total_flos': 855023988768768.0, 'train_loss': 9.732076955396076, 'epoch': 0.9885057471264368})

In [18]:
predictions = trainer.predict(test_dataset) 

In [19]:
import numpy as np

# Handle logits or token IDs
if isinstance(predictions.predictions, tuple):
    preds = predictions.predictions[0]
else:
    preds = predictions.predictions

if preds.ndim == 3:
    preds = np.argmax(preds, axis=-1)

labels = np.where(predictions.label_ids == -100, tokenizer.pad_token_id, predictions.label_ids)

decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


In [22]:
for ref, pred in zip(decoded_labels[:10], decoded_preds[:10]):
    print(f"Actual   : {ref}")
    print(f"Predicted: {pred}")
    print("-" * 100)


Actual   : Moses and Aaron were used to guide the nation and Miriam led the women of Israel in a victory dance
Predicted: Jehovah used Moses and Aaron to guide the nation of Israel, and Miriam to guide the women of Israel in their victorious march.
----------------------------------------------------------------------------------------------------
Actual   : In Kumarakom the monsoon are early June to mid August November to March considered the best months
Predicted: The best months for the weather in Kumarakom are from early June to mid-August and from November to March.
----------------------------------------------------------------------------------------------------
Actual   : And Joshua made peace with them and made a league with them to let them live and the princes of the congregation sware unto them
Predicted: And Joshua made a covenant with them, and made a league with them, to let them live; and the princes of the people swore unto them.
--------------------------------------

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model and tokenizer from Kaggle input directory
model_path = "./final_model"
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    #quantization_config=quant_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

# Set target language
tgt_lang = "eng_Latn"
forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
model.config.forced_bos_token_id = forced_bos_token_id
model.config.use_cache = True

# Load Manipuri test sentences
test_file = "/kaggle/input/emnlp-test-dataset-2025/indicMT_testset2025_release/Category-1/Assamese/as-en.txt"
with open(test_file, "r", encoding="utf-8") as f:
    manipuri_sentences = [line.split("\t")[0].strip() for line in f if line.strip()]

from tqdm import tqdm  

eng_predictions = []
for sentence in tqdm(manipuri_sentences, desc="Translating"):
    inputs = tokenizer(
        f"asm_Beng{sentence}",
        return_tensors="pt",
        truncation=True,
        max_length=256
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=forced_bos_token_id,
        max_length=256,
        num_beams=5
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    eng_predictions.append(decoded)

# Save predictions
with open("myresult.txt", "w", encoding="utf-8") as f:
    for pred in eng_predictions:
        f.write(pred + "\n")


In [25]:
import shutil

# Zip the folder
shutil.make_archive("/kaggle/working/final_model", 'zip', "./final_model")


'/kaggle/working/final_model.zip'